In [2]:
import nltk as nl
import os
from unidecode import unidecode
from glob import glob
from nltk.stem import PorterStemmer
ps = PorterStemmer()


# def process_text(word):
#     word = word.lower()
#     return unidecode(ps.stem(word))

def process_text(word):
    return unidecode(word)

def create_file(path_name, temp_name, step_one=False, return_=False):
    # step_one determines if to just get the IOB component
    file_data = []
    sentence_break = '_S_B_'

    # load all files in given directory into a list
    for file_name in glob(path_name + '/*.tsv'):
        file = []
        with open(file_name, encoding='utf-8') as f:
            for line in f:
                par_line = line[:-1].split('\t')
                if par_line[0] != '-DOCSTART-':
                    if len(par_line) == 1:
                        file.append([sentence_break, 'O'])
                    else:
                        file.append([process_text(par_line[0]), par_line[3]])  # process each word
        file_data.append((file, file_name))
    data_pos = [[sentence_break, sentence_break, 'O']]

    # combine all files into a single list
    for f in file_data:
        sentence = []
        for word in f[0][1:]:
            if word[0] == sentence_break:
                pos = nl.pos_tag([w[0] for w in sentence])
                for ii in range(len(sentence)):
                    sentence[ii][1] = pos[ii][1]
                data_pos += (sentence + [[sentence_break, sentence_break, 'O']])
                sentence = []
            else:
                sentence.append([word[0], '', word[1]])

    # save as new file of all data together
    if os.path.exists(temp_name):  # replace files instead of adding on
        os.remove(temp_name)

    with open(temp_name, 'a') as f:
        for line in data_pos:
            if step_one:
                f.write("{}\t{}\t{}\n".format(line[0], line[1], line[2][0]))
            else:
                if line[2][0] not in ['I', 'O', 'B']:
                    f.write("{}\t{}\t{}\n".format(line[0], line[1], 'O'))
                else:
                    f.write("{}\t{}\t{}\n".format(line[0], line[1], line[2]))
    if return_:
        return data_pos


In [59]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import eli5

# help and inspiration from
# https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html#let-s-use-conll-2002-data-to-build-a-ner-system
# and
# https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2


def file_opener(file_name):
    sentence_list = []
    current_sentence = []
    with open(file_name) as f:
        for line in f:
            word = line.strip().split('\t')
            if word[0] == '_S_B_':
                if len(current_sentence) > 0:
                    sentence_list.append(current_sentence)
                    current_sentence = []
            else:
                current_sentence.append(tuple(word))
    return sentence_list


def feature_extractor(sentence, i):
    word = sentence[i][0]
    pos = sentence[i][1]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'ps.stem(word)': ps.stem(word),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'pos': pos,
        'pos[:2]': pos[:2],
    }
    if i > 0:
        word1 = sentence[i - 1][0]
        pos1 = sentence[i - 1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:ps.stem(word)': ps.stem(word1),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.isdigit()': word1.isdigit(),
            '-1:pos': pos1,
            '-1:pos[:2]': pos1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sentence) - 1:
        word1 = sentence[i + 1][0]
        pos1 = sentence[i + 1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:ps.stem(word)': ps.stem(word1),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:word.isdigit()': word1.isdigit(),
            '+1:pos': pos1,
            '+1:pos[:2]': pos1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sentence_features(sentence):
    return [feature_extractor(sentence, i) for i in range(len(sentence))]


def sentence_labels(sentence):
    return [label for token, pos, label in sentence]


def sentence_tokens(sentence):
    return [token for token, pos, label in sentence]


In [60]:
train_sentences = file_opener("./NER/train")
dev_sentences = file_opener("./NER/dev")

X_train = [sentence_features(s) for s in train_sentences]
y_train = [sentence_labels(s) for s in train_sentences]

X_dev = [sentence_features(s) for s in dev_sentences]
y_dev = [sentence_labels(s) for s in dev_sentences]

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.09684573395986483,
    c2=0.0800864058815976,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)
labels = list(crf.classes_)
labels.remove('O')
y_predicted = crf.predict(X_dev)

# y_pred_flat = []
# y_dev_flat = []
# for x in y_predicted:
#     y_pred_flat += x
# for x in y_dev:
#     y_dev_flat += x
# print(set(y_pred_flat) - set(y_dev_flat))
# print(set(y_pred_flat))
# print(set(y_dev_flat))

f1 = metrics.flat_f1_score(y_dev, y_predicted, average='weighted', labels=labels)
print(f1)
eli5.show_weights(crf)

0.776543591581169


From \ To,O,B-Abiotic_Entity,I-Abiotic_Entity,B-Aggregate_Biotic_Abiotic_Entity,I-Aggregate_Biotic_Abiotic_Entity,B-Biotic_Entity,I-Biotic_Entity,B-Eventuality,I-Eventuality,B-Location,I-Location,B-Quality,I-Quality,B-Time,I-Time,B-Unit,I-Unit,B-Value,I-Value
O,3.074,0.6,-2.173,0.923,-2.626,0.778,-1.987,0.69,-2.686,1.209,-2.594,0.745,-0.586,1.263,-3.63,-0.265,-2.157,2.126,-1.84
B-Abiotic_Entity,0.473,-1.661,6.091,-1.43,-1.452,-1.623,-1.02,0.989,-1.176,0.0,0.0,-0.002,-1.014,0.203,0.0,0.0,0.0,0.0,0.0
I-Abiotic_Entity,-0.236,-0.948,2.361,0.0,-0.041,-0.282,0.0,0.359,-0.286,0.0,0.0,0.274,0.0,0.205,-0.144,0.0,0.0,0.0,0.0
B-Aggregate_Biotic_Abiotic_Entity,0.252,-1.054,-0.124,-1.053,6.054,-0.162,-2.476,0.401,-1.745,1.211,-0.226,-0.426,-1.481,0.563,-0.615,-0.055,-0.063,-0.908,0.0
I-Aggregate_Biotic_Abiotic_Entity,0.307,-0.526,-0.49,-0.572,5.418,-0.176,-0.906,-0.599,-0.298,0.0,0.0,-0.386,-0.165,0.281,-0.872,-0.001,0.0,0.0,0.0
B-Biotic_Entity,0.359,-0.292,-0.98,-0.359,-2.568,-1.231,5.237,1.184,-1.863,-0.611,-0.978,0.177,-2.784,-0.016,-0.783,0.0,-0.558,-0.603,-0.014
I-Biotic_Entity,0.264,-0.999,-0.114,-1.173,-0.741,-1.287,4.088,0.48,0.0,-0.949,-0.984,-0.479,-0.766,-0.214,-0.306,0.0,-0.309,-0.286,0.0
B-Eventuality,0.212,0.996,-1.259,1.212,-2.293,1.109,-2.509,-0.272,4.202,0.3,-0.521,0.008,-2.393,0.4,-3.073,-0.203,-0.905,0.459,-0.334
I-Eventuality,0.569,0.177,-0.045,-0.833,-0.357,0.088,-0.518,-0.664,4.009,-0.232,0.0,-0.522,-0.295,-0.122,-1.856,0.0,0.0,0.031,0.0
B-Location,-1.058,0.454,0.0,0.331,-0.65,0.0,-1.881,-1.089,-0.573,-0.271,5.115,-0.274,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0


# Unified

In [1]:
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

In [52]:


crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  8.2min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=True, averaging=None,
                                 c=None, c1=None, c2=None,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None,...
                   scoring=make_scorer(flat_f1_score, average=weighted, labels=['B-Quality', 'B-Biotic_Entity', 'B-Eventuality', 'I-Biotic_Entity', 'B-Location', 'I-Location', 'B-Time', 'I-Time', 'I-Eventuality', 'B-Value', 'B-Aggregate_Biotic_Abiotic_Entity', 'B-Unit', 'I-Aggregate_Biotic_Abiotic_Entity', 'I-Unit', 'B-Abiotic_Entity', 'I-Abioti

In [53]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.09684573395986483, 'c2': 0.0800864058815976}
best CV score: 0.716436871144859
model size: 1.72M


ModuleNotFoundError: No module named 'file_loader'

In [27]:
from glob import glob

train_files = set([x[21:] for x in glob("./Data/bio-ner/train" + '/*.tsv')])
dev_files = set([x[19:] for x in glob("./Data/bio-ner/dev" + '/*.tsv')])
test_files = set([x[20:] for x in glob("./Data/bio-ner/test" + '/*.tsv')])

In [35]:
train_files.intersection(dev_files)

{'13451175_okra.tsv'}

In [34]:

test_files.intersection(dev_files)

set()

In [ ]:
Final Unified Test Scores
IOB Score: 0.8515465401313206
Class Score: 0.8171233546749452
Overall Score: 0.772362130135599







IOB Score: 0.8471202213420131	Diff:
Class Score: 0.8042069357722814	Diff:
Overall Score: 0.760802348363734	Diff: 

In [5]:
0.8471202213420131 - 0.8515465401313206

-0.004426318789307437

In [6]:
 0.8042069357722814 - 0.8171233546749452

-0.012916418902663818

In [7]:
0.760802348363734 - 0.772362130135599

-0.011559781771865008